In [ ]:
import snscrape.modules.twitter as sntwitter
import os
from datetime import datetime, timedelta

# enter country name
nation = "india"

In [ ]:
import pandas as pd
from stop_words import get_stop_words
from nltk.tokenize import word_tokenize

# Load stopwords
stop_words = get_stop_words('english')

# Function to tokenize and remove stopwords
def tokenize_and_remove_stopwords(text):
    tokens = word_tokenize(text)  # Tokenize the text
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]  # Remove stopwords
    return filtered_tokens

In [ ]:
def find_country_in_text(text, df,df1):
    for word in text:
        wd = word.lower()
        v = df1.iloc[1:,1].searchsorted(wd)
        if(v == 44690):
            continue
        if wd == df1.iloc[v+1,1]:
            return wd
    for word in text:
        wd = word.lower()
        v = df.iloc[1:,0].searchsorted(wd)
        if(v == 44690):
            continue
        if wd == df.iloc[v+1,0]:
            return df.iloc[v+1,1]
    return "none"

# Example usage
df = pd.read_csv("./../worldcities.csv",usecols = [1,4])
df.sort_values("city_ascii", inplace=True)
df = df.reset_index(drop=True)
df["city_ascii"] = df["city_ascii"].apply(lambda x: x.lower())
df["country"] = df["country"].apply(lambda x: x.lower())

df1 = pd.read_csv("./../worldcities.csv",usecols = [1,4])
df1.sort_values("country", inplace=True)
df1 = df1.reset_index(drop=True)
df1["city_ascii"] = df1["city_ascii"].apply(lambda x: x.lower())
df1["country"] = df1["country"].apply(lambda x: x.lower())

In [ ]:
attributes_container = []

keyword = ["sadness","loneliness","Despair","Anguish", "Grief","Anxiety","Stress","Depression","Isolation","Helplessness","happiness","joy","love","gratitude", "hope","success","motivation","inspiration","excitement","optimism"]

tweet_df = pd.DataFrame(attributes_container,columns=["User", "Date Created","Language", "Tweet ID", "Tweet", "location"])
for keys in keyword:
    attributes_container = []
    query = "{} lang:en".format(keys)
    i = 0
    for b,tweet in enumerate(sntwitter.TwitterSearchScraper(query).get_items()):
        tt = tokenize_and_remove_stopwords(tweet.user.location)
        t = find_country_in_text(tt, df,df1)
        if(t == nation):
            if i>=5000:
                i = 0
                break
            attributes_container.append([tweet.user.username, tweet.date,tweet.lang, tweet.id,tweet.rawContent, t])
            i = i + 1
    
    tweets_df = pd.DataFrame(attributes_container, columns=["User", "Date Created","Language", "Tweet ID", "Tweet", "location"])
    tweet_df = pd.concat([tweet_df, tweets_df])

In [ ]:
print(tweet_df["Tweet ID"].size)